In [9]:
import psycopg2
import pandas
import db_config

import json
import urllib.parse
import requests
import time
import matplotlib.pyplot as plt

In [10]:
# import API key from config file
api_key = db_config.api_key

# define database table names
population_table = db_config.population_table
msa_table = db_config.msa_table

In [7]:
def read_data():
    try:
        cursor = connection.cursor()
        query = f"SELECT b.MSA_TITLE, a.YEAR, a.POPULATION FROM {population_table} a JOIN {msa_table} b ON a.MSA_CODE=b.MSA_CODE"
        cursor.execute(query)
        result = cursor.fetchall()
        # Fetch column names from the cursor description
        columns = [desc[0] for desc in cursor.description]
        df = pandas.DataFrame(result,columns=columns)
        print(df)
        # Create a pandas dataframe from the query result and column names
    except (Exception, Error) as error:
        print("Error while executing the query:", error)
       
def drop_data():
    try:
        cursor.execute(""f"DELETE FROM {population_table}""")
        connection.commit()
        print("\nRecords dropped successfully")
    except (Exception, Error) as error:
        print("Error while dropping records", error)
            
def insert_data():
    try:
        cursor = connection.cursor()
        for _, row in population_df.iterrows():
            query = f"INSERT INTO {population_table} (msa_code,Year,population) VALUES (%s, %s, %s)"
            cursor.execute(query, tuple(row))
        connection.commit()
        print("\nData inserted successfully!\n\n")
    except (Exception, Error) as error:
        connection.rollback()
        print("Error while inserting data into PostgreSQL:\n\n", error)
        
def custom_query():
    try:
        query = input('Enter SQL query: ')
        cursor.execute(query)
        result = cursor.fetchall()
            
        # Fetch column names from the cursor description
        columns = [desc[0] for desc in cursor.description]
        df = pandas.DataFrame(result,columns=columns)
        print(df)
        print('\nExecution complete.\n\n')
    except (Exception, Error) as error:
        connection.rollback()
        print("Error while executing query to PostgreSQL.\n\n")  
        
def visualize():
    try:
        cursor = connection.cursor()
        query = f"SELECT b.MSA_TITLE, a.YEAR, a.POPULATION FROM {population_table} a JOIN {msa_table} b ON a.MSA_CODE=b.MSA_CODE"
        cursor.execute(query)
        result = cursor.fetchall()
        # Fetch column names from the cursor description
        columns = [desc[0] for desc in cursor.description]
        df = pandas.DataFrame(result,columns=columns)  
        if df.empty:
            print('\nDataFrame is empty!\n')
        else:
            # Pivot the dataframe to have 'year' as columns and 'metropolitan' as index
            pivot_df = df.pivot(index='year', columns='msa_title', values='population')
            # Plot the grouped bar chart
            pivot_df.plot(kind='bar', stacked=False, figsize=(13,6))
            plt.xlabel('Year')
            plt.ylabel('Population')
            plt.title('Population by MSA and Year')
            plt.legend(title='Year', loc='upper right', bbox_to_anchor=(1, 1))
            plt.ticklabel_format(style='plain', axis='y')
            # Show the plot
            plt.show()
    except (Exception, Error) as error:
        print("Error while visualizing:", error)   

In [12]:
connection = psycopg2.connect(
    host = db_config.host,
    port = db_config.port,
    user = db_config.user,
    password = db_config.password,
    database = db_config.database
    )
cursor=connection.cursor()

while True:
    user_input = input("********************************************************\nMetropolitan Statistical Area (MSA) Population Interface\n********************************************************\n\nSelect one of the following numeric options:\n  1 - Read database table\n  2 - Drop all records\n  3 - Retrieve data from Census API\n  4 - insert into database table\n  5 - open SQL Query Editor (SELECT statements only)\n  6 - Visualizer   \n\ntype EXIT at any time to stop the program\n")

    if user_input.lower() == "exit":
        print("\nClosing connection and exiting the loop...")
        cursor.close()
        connection.close()
        print("\nConnection closed.")
        break

    if user_input == "1":
        # Perform method 1
        print(f"Retrieving data from PostgreSQL db table: {population_table} ...")
        read_data()
        print("\n\n")
        time.sleep(0.5)

    elif user_input == "2":
        # Perform method 2
        print("\n\nDropping all records from table: metropolitan_population ...")
        drop_data()
        print("\n\n")
        time.sleep(0.5)
        
    elif user_input == "3":
        # Perform method 3
        start = int(input("Enter starting year (cannot be later than 2009):"))-1
        end = int(input("Enter ending year (cannot be current year):"))
        try:
            cursor = connection.cursor()
            query = f"SELECT * FROM {msa_table}"
            cursor.execute(query)
            result = cursor.fetchall()
            # Fetch column names from the cursor description
            columns = [desc[0] for desc in cursor.description]
            msa_df = pandas.DataFrame(result,columns=columns)
            # Create a pandas dataframe from the query result and column names 
        except (Exception, Error) as error:
            print("Error while executing the query:", error)
        search_input = input('Search for city to retrieve corresponding MSA name: ')
        search_result = msa_df[msa_df['msa_title'].str.contains(search_input, case=False)]
        print("\n", search_result, "\n")   
        while input('Is this the city you were looking for (yes/no)?: ').lower() != 'yes':
            search_input = input('Search for city to retrieve corresponding MSA name: ')
            search_result = msa_df[msa_df['msa_title'].str.contains(search_input, case=False)]
            print("\n",search_result)

        search_msa = int(input("Enter desired MSA Code (above): "))            

        print("\n\nRetrieving data from Census API...")
        population_df = pandas.DataFrame(columns=['msa_code','Year','Population'])
        for years in range(start,end):
            b = years + 1
            time.sleep(0.025)
            base_url = f'https://api.census.gov/data/{b}/acs/acs5?get=NAME,B01001_001E&for=metropolitan%20statistical%20area/micropolitan%20statistical%20area:{search_msa}&key='
            final_url = base_url + api_key
            json_data = requests.get(final_url).json()
            population = json_data[1][1]
            msa = json_data[1][2]
            insert_row = {'msa_code': msa, 'Year': b, 'Population': population}
            population_df = pandas.concat([population_df, pandas.DataFrame([insert_row])])
        print(population_df, "\n\n")   
        time.sleep(0.5)
        
    elif user_input == "4":
        # Perform method 4
        print(f"\n\nInserting data into database table {population_table}...")
        insert_data()
        time.sleep(0.5)
            
    elif user_input == "5":
        # Perform method 5
        print("\n\nCustom Query Editor (SELECT statements only)...")
        custom_query()
        time.sleep(0.5)
            
    elif user_input == "6":
        # Perform method 6
        print("\n\nVisualizer...")
        visualize()
        time.sleep(0.5)

    else:
        print("Invalid command. Please try again.\n\n")
        time.sleep(0.5)

********************************************************
Metropolitan Statistical Area (MSA) Population Interface
********************************************************

Select one of the following numeric options:
  1 - Read database table
  2 - Drop all records
  3 - Retrieve data from Census API
  4 - insert into database table
  5 - open SQL Query Editor (SELECT statements only)
  6 - Visualizer   

type EXIT at any time to stop the program
 exit



Closing connection and exiting the loop...

Connection closed.
